In [1]:
import numpy as np
import torch

import cvxpy as cp
from cvxpylayers.torch import CvxpyLayer
import plotly.express as px
import pandas as pd
import os

In [2]:

# Set working directory
os.chdir(r"..") # should be the git repo root directory
print("Current working directory: " + os.getcwd())
repo_name = 'net-load-forecasting'
assert os.getcwd()[-len(repo_name):] == "net-load-forecasting", "Working directory is not the git repo root directory"

Current working directory: /Users/nikolaushouben/Desktop/net-load-forecasting


# Learning the Cost Structure

## Generate data

In [4]:
np.random.seed(0)

n_timesteps = 96
n_assets = 1

# Generate some random data for load and production
tariff = np.zeros(n_timesteps)
tariff[20:30] = 0.2
tariff[60:70] = 0.2
peak_charge = 0.0

# load follows a sin curve with noise and a peak in the middle of the day
load = np.sin(np.linspace(0, 2 * np.pi, n_timesteps)) + np.random.normal(0, 0.3, n_timesteps) + 3
load = np.maximum(load, 0)
load[20:30] += np.random.uniform(0, 10, 10)
load[60:70] += np.random.uniform(0, 10, 10)

# production is photovoltaic power output so bell curve with peak in the middle of the day
production = 5*np.exp(-((np.linspace(0, 2.0 * np.pi, n_timesteps) - np.pi) ** 2) / 2) + np.random.normal(0, 0.3, n_timesteps)
production = np.maximum(production, 0)


## Read in Real Data

In [5]:
os.getcwd()

'/Users/nikolaushouben/Desktop/net-load-forecasting'

In [6]:
clean_data_path = os.path.join(os.getcwd(),'data','clean_data')

df_loads = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key='15min/loads') / 1000

df_prods = pd.read_hdf(os.path.join(clean_data_path, "data_net_load_forecasting.h5"), key='15min/pvs') / 1000

In [7]:
df_loads

component,SFH3,SFH4,SFH5,SFH9,SFH10,SFH12,SFH16,SFH18,SFH19,SFH21,SFH22,SFH23,SFH27,SFH28,SFH29,SFH30,SFH31,SFH32,SFH36,SFH38
time,,,,,,,,,,,,,,,,,,,,
2014-08-27 00:00:00,0.111361,0.222306,0.329591,0.215298,0.391152,0.079697,0.061138,0.205449,0.208618,0.162622,0.225695,NaN,0.059182,0.103002,0.133036,0.036228,0.159548,0.127428,0.105634,0.212763
2014-08-27 00:15:00,0.106765,0.221239,0.329594,0.160308,0.341218,0.061602,0.069773,0.210328,0.208757,0.123030,0.219840,NaN,0.123353,0.170560,0.088772,0.036618,0.125464,0.127130,0.133042,0.150705
2014-08-27 00:30:00,0.124059,0.171173,0.329597,0.159689,0.387114,0.139278,0.048935,0.180717,0.222862,0.124311,0.168639,NaN,0.036559,0.226051,0.088758,0.077901,0.132419,0.128093,0.132255,0.228130
2014-08-27 00:45:00,0.041674,0.139017,0.329600,0.216311,0.412822,0.151100,0.066599,0.200785,0.239989,0.258810,0.149001,NaN,0.038361,0.199237,0.090211,0.114029,0.165961,0.128011,0.108528,0.255193
2014-08-27 01:00:00,0.124037,0.139225,0.329603,0.281426,0.453645,0.126606,0.083099,0.232525,0.240246,0.256173,0.147657,NaN,0.129772,0.164446,0.128687,0.107869,0.180287,0.200751,0.117628,0.218715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-26 22:45:00,0.203273,0.381182,0.373536,0.554878,NaN,0.176809,0.184739,0.112115,0.180386,0.190256,0.263751,0.321093,0.100799,0.125384,0.116598,0.057372,0.182863,0.288013,0.103820,0.676926
2016-12-26 23:00:00,0.235491,0.224977,0.351130,0.561532,NaN,0.186430,0.351948,0.122269,0.205957,0.172919,0.250550,0.339433,0.062062,0.158393,0.168175,0.054208,0.175261,0.269624,0.105853,0.552589
2016-12-26 23:15:00,0.249168,0.238192,0.364014,1.004193,NaN,0.176123,0.143297,0.179858,0.202400,0.158950,0.291155,0.331568,0.080251,0.109076,0.168032,0.098792,0.166283,0.168833,0.103448,0.392800


In [8]:
load = df_loads.iloc[:96,0].values
production = df_prods.iloc[:96,0].values

## Energy Model

In [33]:
def get_hems(energy_capacity, power_capacity, timestep):
    
    p_load = cp.Parameter((n_timesteps, n_assets), name="load", nonneg=True)
    p_production = cp.Parameter((n_timesteps, n_assets), nonneg=True, name="production")
    p_tariff = cp.Parameter((n_timesteps, n_assets), name="tariff", nonneg=True)
    p_initial_state_of_energy = cp.Parameter((n_assets,), nonneg=True, name="initial_state_of_energy")

    v_battery_power = cp.Variable((n_timesteps, n_assets), name="battery_power")
    v_grid_power = cp.Variable((n_timesteps, n_assets), name="grid_power")
    v_state_of_energy = cp.Variable((n_timesteps, n_assets), nonneg=True, name="state_of_energy")

    constraints = []
    constraints += [v_state_of_energy <= energy_capacity]
    constraints += [v_state_of_energy[0] == p_initial_state_of_energy]
    constraints += [v_state_of_energy[1:] == v_state_of_energy[:-1] + v_battery_power[:-1] * timestep]
    constraints += [v_battery_power <= power_capacity]
    constraints += [v_battery_power >= -power_capacity]
    constraints += [v_grid_power == v_battery_power + p_load - p_production]
    constraints += [v_battery_power >= -p_load]

    tariff_cost = cp.sum(cp.multiply(p_tariff, cp.maximum(v_grid_power,0)) * timestep)
    
    objective = cp.Minimize(tariff_cost)
    problem = cp.Problem(objective, constraints)
    
    hems = CvxpyLayer(problem, variables=[v_battery_power, v_grid_power, v_state_of_energy], parameters=[p_load, p_production, p_tariff, p_initial_state_of_energy])

    return hems


In [34]:
timestep = 1.
initial_state_of_energy = 5.
energy_capacity = 10
power_capacity = 5
efficiency = 0.9

hems = get_hems(energy_capacity, power_capacity, timestep)

load_input = torch.tensor(load, dtype=torch.float32).reshape(-1, 1)
production_input = torch.tensor(production, dtype=torch.float32).reshape(-1, 1)
tariff_input = torch.tensor(tariff, dtype=torch.float32).reshape(-1, 1)
initial_state_of_energy_input = torch.tensor([initial_state_of_energy], dtype=torch.float32)

battery_power, grid_power, state_of_energy = hems(load_input, production_input, tariff_input, initial_state_of_energy_input)

df_results = pd.DataFrame({
    "load": load_input.detach().flatten(),
    "production": production_input.detach().flatten(),
    "tariff": tariff_input.detach().flatten(),
    "battery_power": battery_power.detach().flatten(),
    "grid_power": grid_power.detach().flatten(),
    "state_of_energy": state_of_energy.detach().flatten()
})


cost = df_results["tariff"]*df_results["grid_power"]*timestep

fig = px.line(df_results, y=["load", "production", "battery_power", "grid_power", "state_of_energy"])

fig.add_scatter(x=np.arange(n_timesteps), y=df_results["tariff"], mode="lines", name="tariff", yaxis="y2")

fig.update_layout(yaxis2=dict(overlaying='y', side='right'), title=f"{cost.sum()}")

## Learning the Load based on Trajectories

In [ ]:
from tqdm.notebook import tqdm

def mse(y_true, y_pred):
    return torch.mean((y_true - y_pred) ** 2)

load_pred = torch.full((n_timesteps, 1), 0, requires_grad=True, dtype=torch.float32)
initial_state_of_energy_pred = torch.full((n_assets,), 5, requires_grad=True, dtype=torch.float32)

opt = torch.optim.Adam([load_pred, initial_state_of_energy_pred ], lr=0.001)
train_losses = []
for i in tqdm(range(10000)):
    opt.zero_grad()
    _, grid_power_pred, _ = hems(load_pred, production_input, tariff_input, initial_state_of_energy_pred)
    loss = mse(grid_power, grid_power_pred)
    
    loss.backward()
    train_losses.append(loss.item())
    opt.step()

    if len(train_losses) % 100 == 0:
        print(f"Training loss: {loss.item()}")
        opt.param_groups[0]["lr"] *= 0.96


In [52]:
px.line(x=np.arange(len(train_losses)), y=train_losses)

In [56]:
fig = px.line(torch.concat([load_pred, load_input], axis=1).detach().numpy(), title="Load")

fig.show()

In [57]:
px.line(np.concatenate([grid_power.detach().numpy(), grid_power_pred.detach().numpy()], axis=1), title="Grid Power")

In [58]:
initial_state_of_energy_pred, initial_state_of_energy_input

(tensor([4.9913], requires_grad=True), tensor([5.]))

In [ ]:
from darts.models import TiDEModel